## Satisfaction Analysis - Task 4

In [1]:
# Import packages
import os,sys

# Add the path to the directory
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)
    
from scripts.utils import *
from db_connection.postgresql_connection import *

In [2]:
# Load dataset from postgres
telecom_df = fetch_data_from_postgres()


In [3]:
# Dataframe head
telecom_df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [4]:
# filling missing value with mean or mode
fill_missing_values_t3(telecom_df)


In [50]:
# process df
telecom_df['Total_Avg_RTT'] = telecom_df['Avg RTT DL (ms)'] + telecom_df['Avg RTT UL (ms)']
telecom_df['Total_Avg_Bearer_TP'] = telecom_df['Avg Bearer TP DL (kbps)'] + telecom_df['Avg Bearer TP UL (kbps)']
telecom_df['Total_Avg_TCP'] = telecom_df['TCP DL Retrans. Vol (Bytes)'] + telecom_df['TCP UL Retrans. Vol (Bytes)']
telecom_df['Total TCP Retrans. Vol (Bytes)'] = telecom_df['TCP DL Retrans. Vol (Bytes)'] + telecom_df['TCP UL Retrans. Vol (Bytes)']

# user experience scaled
user_experience = telecom_df.set_index('MSISDN/Number')[
     ['Total_Avg_RTT', 'Total_Avg_Bearer_TP', 'Total TCP Retrans. Vol (Bytes)']]
scaler = StandardScaler()
scaled_array = scaler.fit_transform(user_experience)
pd.DataFrame(scaled_array).head(5)


,0,1,2
0,-0.142369,-0.559296,3.145329e-17
1,-0.101671,-0.560227,3.145329e-17
2,0.000000,-0.561234,3.145329e-17
3,0.000000,-0.558513,3.145329e-17
4,0.000000,-0.561234,3.145329e-17


In [43]:
user_experience

,Total_Avg_RTT,Total_Avg_Bearer_TP,Total TCP Retrans. Vol (Bytes)
MSISDN/Number,,,
3.366496e+10,47.000000,67.000000,2.156957e+07
3.368185e+10,70.000000,42.000000,2.156957e+07
3.376063e+10,127.458589,15.000000,2.156957e+07
3.375034e+10,127.458589,88.000000,2.156957e+07
3.369980e+10,127.458589,15.000000,2.156957e+07
...,...,...,...
3.365069e+10,32.000000,117.000000,2.156957e+07
3.366345e+10,29.000000,77.000000,2.156957e+07
3.362189e+10,49.000000,90.000000,2.156957e+07


In [51]:
# load user engagement 

user_engagement = pd.read_csv("../data/user_engagement.csv")

Task 4.1

Engagement score calculated and assigned


Assign experience score to each user

In [52]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(user_engagement)
scaled_data

array([[-3.99191484e-03, -2.08088240e-01, -4.89228047e-01, ...,
         3.80796440e-01,  1.55416341e+00,  1.55416341e+00],
       [-3.99191483e-03,  9.93460814e-01, -4.89228047e-01, ...,
        -1.08775937e+00, -2.62956868e-01, -2.62956868e-01],
       [-3.99191457e-03,  9.93460814e-01, -4.89228047e-01, ...,
        -1.94402111e-01, -2.62956868e-01, -2.62956868e-01],
       ...,
       [ 1.08886651e+00, -1.40963729e+00, -4.89228047e-01, ...,
        -9.34635054e-01,  6.45603272e-01,  6.45603272e-01],
       [ 1.16406925e+02,  9.93460814e-01, -4.89228047e-01, ...,
        -1.93748347e-01,  6.45603272e-01,  6.45603272e-01],
       [ 3.04823830e+02, -1.40963729e+00, -4.89228047e-01, ...,
        -1.12378199e+00,  6.45603272e-01,  6.45603272e-01]])

In [53]:
# normalize data

normalized_data = normalize(scaled_data)
normalized_data

array([[-0.00173658, -0.0905235 , -0.21282622, ...,  0.1656558 ,
         0.67609926,  0.67609926],
       [-0.00248413,  0.61822022, -0.30444147, ..., -0.67690122,
        -0.16363529, -0.16363529],
       [-0.00336625,  0.83775169, -0.41254936, ..., -0.16393268,
        -0.22174258, -0.22174258],
       ...,
       [ 0.24070047, -0.31160877, -0.10814679, ..., -0.20660668,
         0.14271447,  0.14271447],
       [ 0.9999106 ,  0.00853362, -0.00420236, ..., -0.00166426,
         0.00554559,  0.00554559],
       [ 0.99989588, -0.00462395, -0.00160479, ..., -0.00368628,
         0.00211773,  0.00211773]])

Engagement_score

In [ ]:
import pickle

with open("../models/user_engagement_cluster.pkl", "rb") as f:
    k_means1 = pickle.load(f)

In [55]:
less_engaged_cluster = 3

distance = k_means1.fit_transform(normalized_data)
distance_from_less_engaged_cluster = list(
    map(lambda x: x[less_engaged_cluster], distance))
user_engagement['engagement_score'] = distance_from_less_engaged_cluster
user_engagement

,MSISDN/Number,Cluster,sessions,time_duration,Total_UL_and_DL,cluster,best_cluster,engagement_score
0,3.360100e+10,1,1,116720.0,8.786906e+08,3,3,0.971212
1,3.360100e+10,2,1,181230.0,1.568596e+08,1,1,0.920024
2,3.360100e+10,2,1,134969.0,5.959665e+08,1,1,0.979485
3,3.360101e+10,2,1,49878.0,4.223207e+08,1,1,1.082198
4,3.360101e+10,0,2,37104.0,1.457411e+09,0,0,1.924275
...,...,...,...,...,...,...,...,...
106466,3.379000e+10,1,1,8810.0,7.146416e+08,3,3,1.026010
106467,3.379000e+10,2,1,140988.0,4.803073e+08,1,1,0.947296
106468,3.197021e+12,0,1,877385.0,2.321240e+08,2,2,1.173454
106469,3.370000e+14,2,1,253030.0,5.962878e+08,2,2,1.374102


Experience score

In [ ]:
import pickle

with open("../models/user_exp_cluster.pkl", "rb") as f:
    k_means2 = pickle.load(f)

In [61]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(user_experience)
scaled_data

array([[-1.42368580e-01, -5.59295518e-01,  3.14532900e-17,
         6.64852571e-01],
       [-1.01670907e-01, -5.60227461e-01,  3.14532900e-17,
         6.78572912e-01],
       [ 0.00000000e+00, -5.61233960e-01,  3.14532900e-17,
         6.99399952e-01],
       ...,
       [-1.38829652e-01, -5.58438130e-01,  3.14532900e-17,
         6.66068555e-01],
       [-1.51215900e-01, -5.59146407e-01,  3.14532900e-17,
         6.61522367e-01],
       [ 0.00000000e+00,  6.78078011e-17,  3.14532900e-17,
        -6.71516769e-01]])

In [62]:
# normalize data
normalized_data = normalize(scaled_data)
normalized_data

array([[-1.61708444e-01, -6.35272249e-01,  3.57260190e-17,
         7.55168556e-01],
       [-1.14777797e-01, -6.32449101e-01,  3.55080862e-17,
         7.66051038e-01],
       [ 0.00000000e+00, -6.25859818e-01,  3.50751233e-17,
         7.79935567e-01],
       ...,
       [-1.57722872e-01, -6.34435544e-01,  3.57337440e-17,
         7.56713311e-01],
       [-1.71978129e-01, -6.35918265e-01,  3.57718862e-17,
         7.52350638e-01],
       [ 0.00000000e+00,  1.00977078e-16,  4.68391728e-17,
        -1.00000000e+00]])

In [63]:
less_experience_cluster = 1

distance = k_means2.fit_transform(normalized_data)
distance_from_less_experience_cluster = list(
    map(lambda x: x[less_experience_cluster], distance))
user_experience['experience_score'] = distance_from_less_experience_cluster
user_experience

,Total_Avg_RTT,Total_Avg_Bearer_TP,Total TCP Retrans. Vol (Bytes),experience_score
MSISDN/Number,,,,
3.366496e+10,47.000000,67.000000,2.156957e+07,1.902225
3.368185e+10,70.000000,42.000000,2.156957e+07,1.906573
3.376063e+10,127.458589,15.000000,2.156957e+07,1.912085
3.375034e+10,127.458589,88.000000,2.156957e+07,1.912219
3.369980e+10,127.458589,15.000000,2.156957e+07,1.912085
...,...,...,...,...
3.365069e+10,32.000000,117.000000,2.156957e+07,1.898714
3.366345e+10,29.000000,77.000000,2.156957e+07,1.897869
3.362189e+10,49.000000,90.000000,2.156957e+07,1.902696
